In [1]:
!pip install gradio langchain langchain_community langchain_milvus transformers pymilvus

In [2]:
!pip install replicate

In [3]:
!pip install sentence-transformers

In [4]:
import os
import gradio as gr
import tempfile
from langchain_milvus import Milvus
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# Replicate token and model setup (use your token here)
os.environ['REPLICATE_API_TOKEN'] = ""
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(model=model_path, replicate_api_token=os.environ['REPLICATE_API_TOKEN'])

# Temporary Milvus DB for vector storage
db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

# Write your chemistry sample document to a file for loading
filename = "/content/chemistry_acids_bases.txt"
with open(filename, "w") as f:
    f.write("""
Acids and Bases Chemistry Lab Manual

Theory:
- Acids donate H⁺ (hydrogen ions); bases donate OH⁻ (hydroxide ions).
- Acid-base titration is used to determine the concentration of solutions.

Objective:
- To determine the concentration of acid or base using titration.

Apparatus:
- Burette, pipette, conical flask, beakers, indicators (phenolphthalein, methyl orange), acid and base solutions.

Procedure:
1. Rinse the burette with standard base. Fill it.
2. Pipette a measured amount of acid into a conical flask. Add indicator.
3. Titrate the acid by adding base slowly, swirling, until endpoint color change.
4. Record the burette reading.
5. Repeat for better accuracy.

Observations:
- Note initial and final burette readings.
- Volume of base used = Final - Initial readings.
- Record color change.

Calculations:
- Use normality equation: N1V1 = N2V2.
- Substitute known values to find unknown strength.

Precautions:
- Read burette at eye level.
- Add base dropwise near endpoint.
- Use white tile to observe color change.

Sample Viva Q&A:
Q: What is a neutralization reaction? A: Acid and base react to form salt and water.
Q: Why is phenolphthalein pink in base? A: It is colorless in acid, turns pink in base.
""")

# Load & split content using your tokenizer & splitter
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=tokenizer.model_max_length // 2,
    chunk_overlap=0,
)
texts = splitter.split_documents(documents)

# Add doc_id metadata for traceability in Milvus
for i, doc in enumerate(texts):
    doc.metadata["doc_id"] = i + 1

# Add split documents to Milvus vector store
vector_db.add_documents(texts)

# New prompt tailored for Chemistry content with simple, clear explanations
template = """
You are a helpful Chemistry Lab Assistant specialized in acids and bases.
User Question: {question}
Provide a clear, simple explanation suitable for students.
Include safety tips and procedural guidance when relevant.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Setup chains with your Replicate LLM and prompt
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

# Gradio interface function to ask questions
def ask_chemistry_agent(query):
    if not query.strip():
        return "Please enter a question about acids and bases."
    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Define Gradio interface adapted for Chemistry questions
iface = gr.Interface(
    fn=ask_chemistry_agent,
    inputs=gr.Textbox(label="Ask a question about Acids and Bases Chemistry", placeholder="e.g. What is a neutralization reaction?"),
    outputs=gr.Textbox(label="Answer"),
    title="AI Agent for Acids and Bases Chemistry",
    description="Ask questions related to acids and bases lab manual, procedures, theory, and safety tips.",
    theme="default"
)

# Launch interface with sharing enabled
iface.launch(share=True)
